# MNIST Data using CNN

In [ ]:
import tensorflow as tf

/home/alvaro/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

## Helper functions

In [ ]:
# INIT WEIGHTS
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [ ]:
# INIT BIAS
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

Create a 2D convolution using builtin conv2d from TF. From those docs:

Computes a 2-D convolution given 4-D `input` and `filter` tensors.

Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
and a filter / kernel tensor of shape
`[filter_height, filter_width, in_channels, out_channels]`, this op
performs the following:

1. Flattens the filter to a 2-D matrix with shape
   `[filter_height * filter_width * in_channels, output_channels]`.
2. Extracts image patches from the input tensor to form a *virtual*
   tensor of shape `[batch, out_height, out_width,
   filter_height * filter_width * in_channels]`.
3. For each patch, right-multiplies the filter matrix and the image patch
   vector.


In [ ]:
# CONV2D
def conv2d(x, W):
    """
    x -- input tensor with shape [batch, Height, Width, Channels]
    W -- kernel [filter Height, filter W, Channels IN, Channels Out]
    """
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

Create a max pooling layer, again using built in TF functions:

Performs the max pooling on the input.

    Args:
      value: A 4-D `Tensor` with shape `[batch, height, width, channels]` and
        type `tf.float32`.
      ksize: A list of ints that has length >= 4.  The size of the window for
        each dimension of the input tensor.
      strides: A list of ints that has length >= 4.  The stride of the sliding
        window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. 

In [ ]:
# POOLING
def max_pooling_2by2(x):
    """
    x -- input tensor with shape [batch, Height, Width, Channels] : follow this in ksize and strides for reference.
    """
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [ ]:
# Convolutional Layer
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    
    return tf.nn.relu(conv2d(input_x, W) + b)

In [ ]:
# Normal Layer
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [ ]:
# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
# Layers
x_image = tf.reshape(x, [-1, 28, 28, 1])

Off we go!

In [ ]:
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32]) #32 features for each 5x5 batch.
convo_1_pooling = max_pooling_2by2(convo_1)

In [ ]:
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32, 64]) 
# 5 by 5 before, the input will be 32 from the previous layer, output will be 64

convo_2_pooling = max_pooling_2by2(convo_2)

In [ ]:
convo_2_flat=tf.reshape(convo_2_pooling, [-1, 7*7*64]) # by 7 image for 64 features
full_layer_1 = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [ ]:
# DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_1, keep_prob=hold_prob)

In [ ]:
y_predicted = normal_full_layer(full_one_dropout, size=10)

In [ ]:
# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_predicted))

In [ ]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()
steps = 500

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict={x: batch_x, y_true: batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            print("On step: {}".format(i))
            print('Accuracy: ')
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            
            accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(accuracy, feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))
            print('\n')